In [ ]:
## this notebook will be used to verify that our implementation of Sorcha
## with nongravitational accelerations works with known comets

In [1]:
import os
import sys
import numpy as np
import pandas as pd
import pylab as plt

import subprocess
from scipy.interpolate import interp1d
from astroquery.jplhorizons import Horizons
from astroquery.jplsbdb import SBDB

In [65]:
## define our filepaths and filename stems

## filepath for sorcha input and output files
sfpath = "/home/ellie/research/lsst/sorcha_output/3iatlas/"

## filename stems for the orbit files and other files
fname_orb =  "3iatlas_orb" 
fname_stem = "3iatlas" 

## object ID
obj_id = "3iatlas" #"atlas_c2017" #"panstarrs" #

## location of Sorcha config file
config_fpath = "/home/ellie/research/lsst/sorcha_output/3iatlas/Rubin_full_footprint.ini"

## location of pointing database file for Sorcha
pointing_db_path = "/home/ellie/research/lsst/sorcha_output/3iatlas/lsstcam_20250930.db" 

## location of Earth location file for HelioLinC
earth1day_path = '/home/ellie/research/lsst/heliolinc_files/Earth1day2020s_02a.csv'

## location of ObsCodes file for HelioLinC
obscodes_path = '/home/ellie/research/lsst/heliolinc_files/ObsCodes.html'

## location of colformat file for HelioLinC
colformat_path = '/home/ellie/research/lsst/heliolinc_files/colformat.txt'

## location of hypothesis file for HelioLinC
hypo_path = '/home/ellie/research/lsst/sorcha_output/127005pratchett_new/hihyp00b_mb.txt'

## location of kep2cart.py program
c2c_path = '/home/ellie/research/lsst/lsst_seti/nongrav/sorcha_ng'

## location of kep2cart.py program
k2c_path = '/home/ellie/research/lsst/lsst_seti/nongrav/sorcha_ng'

## import the kep2cart module
com_dir = os.path.abspath(c2c_path)
sys.path.insert(0, com_dir)

from com2cart import com2cart as c2c
from kep2cart import kep2cart as k2c

In [66]:
start_mjd = 60755
#start_jd = 2400000.5+60755

#start_mjd = 60300.0
start_jd = start_mjd+2400000.5
obj_ids = ['3I'] #'DES=  1003219'] #3524'] #, 'DES=  1000025']

ids = []
a = []
q = [] ## periapsis distance
e = []
inc = []
node = []
arg_peri = []
tp = [] ## time of perihelion crossing
ma = []

H = []
G = []

for i in range(len(obj_ids)):
    obj = Horizons(id=obj_ids[i], location='500@10',epochs=start_jd)
    el = obj.elements()
    #print(el['a'][0])

    id = el['targetname'][0].split()[0]    
    ids.append(id)

    a.append(float(el['a'][0]))
    q.append(float(el['q'][0]))
    e.append(float(el['e'][0]))
    inc.append(float(el['incl'][0]))
    node.append(float(el['Omega'][0]))
    arg_peri.append(float(el['w'][0]))
    tp.append(float(el['Tp_jd'][0])-2400000.5)
    ma.append(float(el['M'][0]))
    #H.append(float(el['H'][0]))
    #G.append(float(el['G'][0]))

df_obj = pd.DataFrame()

df_obj['ObjID'] = ids
df_obj['FORMAT'] = ['COM']*len(obj_ids)
df_obj['q'] = q
#df_obj['a'] = a
df_obj['e'] = e
df_obj['inc'] = inc 
df_obj['node'] = node
df_obj['argPeri'] = arg_peri
#df_obj['ma'] = ma
df_obj['t_p_MJD_TDB'] = tp
df_obj['epochMJD_TDB'] = [start_mjd]*len(obj_ids)

df_obj.to_csv("{0}{1}_com.csv".format(sfpath, fname_orb), index=False)

In [67]:
c2c("{0}{1}_com.csv".format(sfpath, fname_orb), "{0}{1}_cart.csv".format(sfpath, fname_orb))

New Cartesian orbits file /home/ellie/research/lsst/sorcha_output/3iatlas/3iatlas_orb_cart.csv written.


In [69]:
## run sorcha

subprocess.run(["sorcha","run","-c", config_fpath,"-p","{0}{1}_phy.csv".format(sfpath, fname_stem),"--orbits",\
                "{0}{1}_nongrav.csv".format(sfpath, fname_orb),"--pd",pointing_db_path,"-o",sfpath,"-t",sfpath+obj_id,\
                "--st","{0}{1}_stats".format(sfpath,obj_id)])

1.0
1.0


CompletedProcess(args=['sorcha', 'run', '-c', '/home/ellie/research/lsst/sorcha_output/3iatlas/Rubin_full_footprint.ini', '-p', '/home/ellie/research/lsst/sorcha_output/3iatlas/3iatlas_phy.csv', '--orbits', '/home/ellie/research/lsst/sorcha_output/3iatlas/3iatlas_orb_nongrav.csv', '--pd', '/home/ellie/research/lsst/sorcha_output/3iatlas/lsstcam_20250930.db', '-o', '/home/ellie/research/lsst/sorcha_output/3iatlas/', '-t', '/home/ellie/research/lsst/sorcha_output/3iatlas/3iatlas', '--st', '/home/ellie/research/lsst/sorcha_output/3iatlas/3iatlas_stats'], returncode=0)

In [ ]:
## read in the gravity-only data: 

df = pd.read_csv(sfpath+obj_id+'.csv')
df = df[df['ObjID'] == '3I_ATLAS_GRAV']

ra_grav = df[

In [ ]:
orb_fname = '/home/ellie/research/lsst/sorcha_output/top20_ng/nongrav_top20_orb.csv'

df_ids = pd.read_csv(orb_fname)
df = pd.read_csv(sorcha_fname)
    
    id_mask = df_ids['ObjID'].str.contains("ONLY_GRAV")
    df_ids_masked = df_ids[~id_mask]
    ids = df_ids_masked['ObjID']
    
    hdist = np.sqrt(df_ids_masked['x']**2 + df_ids_masked['y']**2 + df_ids_masked['z']**2)
    
    mjd_nongrav_lists = []
    ra_nongrav_lists = []
    dec_nongrav_lists = []
    
    mjd_grav_lists = []
    ra_grav_lists = []
    dec_grav_lists = []
    
    for i in range(len(ids)):
    
        id = ids[i]
        d = hdist[i]
    
        mask1 = df['ObjID'] == id
        df_masked = df[mask1]
        
        mask2 = df['ObjID'] == str(id)+"_ONLY_GRAV"
        df_masked_g = df[mask2]
        
        mjd = df_masked['fieldMJD_TAI']
        ra = df_masked['RA_deg']
        dec = df_masked['Dec_deg']
        
        mjd_nongrav_lists.append(mjd)
        ra_nongrav_lists.append(ra)
        dec_nongrav_lists.append(dec)
        
        mjd_g = df_masked_g['fieldMJD_TAI']
        ra_g = df_masked_g['RA_deg']
        dec_g = df_masked_g['Dec_deg']
        
        mjd_grav_lists.append(mjd_g)
        ra_grav_lists.append(ra_g)
        dec_grav_lists.append(dec_g)
        
        if len(ra) != 0:
        
            mjd = np.array(mjd)
            mjd_g = np.array(mjd_g)
            
            fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(12, 8))
        
            axes[0,0].plot(mjd, ra, label=id)
            axes[0,0].plot(mjd_g, ra_g, label=str(id)+"_ONLY_GRAV")
            axes[0,0].set_xlabel("MJD TAI")
            axes[0,0].set_ylabel("RA (deg)")
            axes[0,0].set_title("RA vs MJD")
        
            axes[0,1].plot(mjd, dec, label=id)
            axes[0,1].plot(mjd_g, dec_g, label=str(id)+"_ONLY_GRAV")
            axes[0,1].set_xlabel("MJD TAI")
            axes[0,1].set_ylabel("Dec (deg)")
            axes[0,1].set_title("Dec vs MJD")
            
            ## determine endpoints of interpolation:
            if mjd[0] < mjd_g[0]:
                mjd_min = mjd_g[0]
            else:
                mjd_min = mjd[0]
                
            if mjd[-1] > mjd_g[-1]:
                mjd_max = mjd_g[-1]
            else:
                mjd_max = mjd[-1]
                
            ## determine how many points to interpolate
            ## for, and also create the interpolation
            ## x-dataset
            
            n_interp = len(mjd)
            mjd_new = np.linspace(mjd_min, mjd_max, 100) #n_interp)
            
            ## interpolate RA for both nongrav and only-grav
            ## cases, so we can take the difference
            
            ra_cubic = interp1d(mjd, ra, kind='cubic')
            ra_interp = ra_cubic(mjd_g)
            
            ra_g_cubic = interp1d(mjd_g, ra_g, kind='cubic')
            ra_g_interp = ra_g_cubic(mjd_new)
            
            ## interpolate Dec for both nongrav and only-grav
            ## cases, so we can take the difference
            
            dec_cubic = interp1d(mjd, dec, kind='cubic')
            dec_interp = dec_cubic(mjd_g)#_new)
            
            dec_g_cubic = interp1d(mjd_g, dec_g, kind='cubic')
            dec_g_interp = dec_g_cubic(mjd_new)
            
            axes[1,0].plot(mjd_g, ra_interp-ra_g)#_interp)
            axes[1,0].set_xlabel("MJD TAI")
            axes[1,0].set_ylabel("Delta RA (deg)")
            axes[1,0].set_title("Difference in RA for nongrav vs grav-only")
        
            axes[1,1].plot(mjd_g, dec_interp-dec_g)#_interp)
            axes[1,1].set_xlabel("MJD TAI")
            axes[1,1].set_ylabel("Dec (deg)")
            axes[1,1].set_title("Dec vs MJD")
        
            axes[0,0].legend()
            axes[0,1].legend()
            
            plt.tight_layout()
            #plt.title("{0}, heliocentric dist. = {1} AU".format(id, d))
            plt.show()